In [7]:
# libs
import json
import pandas as pd
import plotly.express as px
from string import punctuation
import datetime
import re


In [8]:
with open('../data/events.json','r') as f:
    event_data = json.loads(f.read())
# Flatten data
events = pd.json_normalize(event_data, record_path =['events'])

with open('../data/meta.json','r') as f:
    meta_data = json.loads(f.read())
# Flatten data
meta = pd.json_normalize(meta_data, record_path =['meta'])

events = events.merge(meta, on='productid')


In [9]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387656 entries, 0 to 387655
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   event        387656 non-null  object
 1   sessionid    387656 non-null  object
 2   eventtime    387656 non-null  object
 3   price        387650 non-null  object
 4   productid    387650 non-null  object
 5   brand        255805 non-null  object
 6   category     387650 non-null  object
 7   subcategory  387650 non-null  object
 8   name         387650 non-null  object
dtypes: object(9)
memory usage: 29.6+ MB


In [10]:
events["eventtime"] = pd.to_datetime(events["eventtime"])
events['price'] = events['price'].astype(float)
events.sort_values("eventtime", inplace=True)


In [11]:
events.isnull().sum()

event               0
sessionid           0
eventtime           0
price               6
productid           6
brand          131851
category            6
subcategory         6
name                6
dtype: int64

In [12]:
events.sessionid.value_counts()

08a906d4-4999-403c-a334-d296106d49cf    308
724a0c30-db26-4cc4-9ee3-e36a663aa43e    287
25370f38-cfd0-4892-93bc-cc1b5b40fa2e    249
174f271c-c13d-48e3-881c-746637fc2002    226
3bb95653-7b91-49dc-ad5e-0f0891d366e5    189
                                       ... 
42701a76-959b-4ff0-9f17-456a87917e20      1
5b5d230d-81af-4765-9eeb-42045d36def4      1
83a3f9bb-8fea-4079-b0c0-0401173ae2a2      1
e4a2b406-a9c9-4a2e-865d-80982a85c711      1
0fa73240-cbfe-43c3-93ab-a8a17673bb26      1
Name: sessionid, Length: 54442, dtype: int64

In [13]:
events.productid.value_counts()

HBV00000NVZGU    17082
HBV00000NVZBI     5557
HBV00000OE7X7     5070
HBV00000NVZBY     3824
HBV00000O2S62     3704
                 ...  
HBV00000OE7BY        1
HBV00000PQJNL        1
OFISBIC829773        1
HBV00000PLGWO        1
HBV00000PVC3D        1
Name: productid, Length: 10235, dtype: int64

In [14]:
events['month'] = pd.DatetimeIndex(events['eventtime']).month
events['weekday'] = pd.Series(events['eventtime']).dt.day_name()
events['hour'] = events['eventtime'].dt.hour
events

,event,sessionid,eventtime,price,productid,brand,category,subcategory,name,month,weekday,hour
218353,cart,2a3cd2f0-683a-4904-bda2-80b04aab06a4,2020-02-10 10:03:40.351000+00:00,16.88,HBV00000NFGV1,Banvit,"Et, Balık, Şarküteri",Kümes Hayvanları,"Banvit Poşetli Bütün Piliç 1,5 kg",2,Monday,10
199732,cart,007af6d7-ef40-4e59-a10b-8b16a33714c4,2020-02-14 18:44:46.001000+00:00,39.99,HBV00000NVZAK,None,"Et, Balık, Şarküteri",Balık ve Deniz Mahsülleri,Çipura (800 g-1000 g),2,Friday,18
195537,cart,4e5e0c4c-3109-44ac-8dfe-069226ae938e,2020-02-21 18:34:20.644000+00:00,1.00,HBV00000NE0TW,Carrefour,Temel Gıda,"Bakliyat, Pirinç, Makarna",Carrefour Gönen Baldo Pirinç 1 kg,2,Friday,18
21292,cart,3a4bc227-b718-443d-802e-c59b1003434b,2020-02-24 20:48:35.901000+00:00,5.45,HBV00000PKHK4,İçim,Kahvaltılık ve Süt,Süt,İçim Rahat Laktozsuz Süt 1 L,2,Monday,20
387055,cart,f8aed59c-e8d5-43a3-ac78-71be39b974a0,2020-02-26 09:57:43.617000+00:00,2.64,HBV00000OE80M,None,Meyve ve Sebze,Meyve,Portakal Finike Gurme 500 gr,2,Wednesday,9
...,...,...,...,...,...,...,...,...,...,...,...,...
336827,cart,c8eb875e-5ec6-4b13-8a1c-bac4bffe849f,2020-07-01 06:08:01.973000+00:00,15.50,HBV00000QX1W5,Felix,Pet Shop,Kedi,Felix Balıklı 4 x 100 gr,7,Wednesday,6
336828,cart,c8eb875e-5ec6-4b13-8a1c-bac4bffe849f,2020-07-01 06:08:17.001000+00:00,15.50,HBV00000QX1W5,Felix,Pet Shop,Kedi,Felix Balıklı 4 x 100 gr,7,Wednesday,6
168730,cart,23c76d91-86e3-4fba-9006-7768dfe0fc19,2020-08-01 06:57:05.337000+00:00,1.25,HBV00000OEL88,None,Fırın,Ekmekler,Normal Ekmek 200 gr,8,Saturday,6
168732,cart,23c76d91-86e3-4fba-9006-7768dfe0fc19,2020-08-01 06:58:08.085000+00:00,1.25,HBV00000OEL88,None,Fırın,Ekmekler,Normal Ekmek 200 gr,8,Saturday,6


In [16]:
session_cnt_by_prdct = pd.DataFrame({'session_cnt_by_prdct': events.groupby(['productid'])['sessionid'].count()})
session_cnt_by_prdct = session_cnt_by_prdct.sort_values("session_cnt_by_prdct", ascending=False).reset_index()
session_cnt_by_prdct = session_cnt_by_prdct.head(50)

import plotly.express as px
fig = px.bar(session_cnt_by_prdct, x='productid', y='session_cnt_by_prdct', title='Product AddToCart Session Count Top 50')
fig.show()

In [17]:
session_cnt_by_category = pd.DataFrame({'session_cnt_by_category': events.groupby(['category', 'subcategory'])['sessionid'].count()})
session_cnt_by_category = session_cnt_by_category.sort_values("session_cnt_by_category", ascending=False).reset_index()

fig = px.bar(session_cnt_by_category, x='category', y='session_cnt_by_category', color='subcategory', title='Category AddToCart Session Count')
fig.show()

In [18]:
category_cnt_by_weekday = pd.DataFrame({'category_cnt_by_weekday': events.groupby(['weekday','category'])['category'].count()})
category_cnt_by_weekday = category_cnt_by_weekday.sort_values("category_cnt_by_weekday", ascending=True).reset_index()

fig = px.bar(category_cnt_by_weekday, x="weekday", y="category_cnt_by_weekday", color='category', title='Category Count Based on Weekday')
fig.show()

In [19]:
category_cnt_by_hour = pd.DataFrame({'category_cnt_by_hour': events.groupby(['hour','category'])['category'].count()})
category_cnt_by_hour = category_cnt_by_hour.sort_values("category_cnt_by_hour", ascending=True).reset_index()

fig = px.bar(category_cnt_by_hour, x="hour", y="category_cnt_by_hour", color='category', title='Category Count Based on Hour')
fig.show()